In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
path = '/Users/timothypan/Documents/Personal Projects/HDB RESALE Prices/ResaleFlatPrices'
# List all CSV files in the directory
all_files = [f for f in os.listdir(path) if f.endswith('.csv')]
print("Files found:", all_files)


Files found: ['ResaleFlatPricesBasedonApprovalDate2000Feb2012.csv', 'ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv', 'ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv', 'ResaleFlatPricesBasedonRegistrationDateFromMar2012toDec2014.csv', 'ResaleFlatPricesBasedonApprovalDate19901999.csv']


In [3]:

dataframes = []
# Loop over each file and load it 
for file in all_files:
    full_path = os.path.join(path, file)
    df = pd.read_csv(full_path)
    dataframes.append(df)

# Concatenate all DataFrames
df = pd.concat(dataframes, ignore_index=True)
df['month'] = pd.to_datetime(df['month'])

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 933423 entries, 0 to 933422
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   month                933423 non-null  datetime64[ns]
 1   town                 933423 non-null  object        
 2   flat_type            933423 non-null  object        
 3   block                933423 non-null  object        
 4   street_name          933423 non-null  object        
 5   storey_range         933423 non-null  object        
 6   floor_area_sqm       933423 non-null  float64       
 7   flat_model           933423 non-null  object        
 8   lease_commence_date  933423 non-null  int64         
 9   resale_price         933423 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 71.2+ MB


In [5]:

# Convert categorical variables to numerical (one-hot encoding)
df = pd.get_dummies(df, columns=['town', 'flat_type', 'flat_model','storey_range'])

# Extract year and month from the 'month' column
df['year'] = df['month'].dt.year
df['month'] = df['month'].dt.month

# Calculate the lease remaining
df["lease_remaining"] = 99 - (df['year'] - df["lease_commence_date"])

# Drop columns that are not needed or that have been converted
df.drop(['block', 'street_name', 'lease_commence_date'], axis=1, inplace=True)

# Reorder columns 
cols = ['year', 'month', 'lease_remaining', 'resale_price'] + [col for col in df.columns if col not in ['year', 'month', 'lease_remaining', 'resale_price']]
df = df[cols]

# Sort the DataFrame by 'year' and 'month'
df.sort_values(by=['year', 'month'], inplace=True)




In [6]:
# Scaling Features
scaler = StandardScaler()
df[['floor_area_sqm', 'lease_remaining']] = scaler.fit_transform(df[['floor_area_sqm', 'lease_remaining']])

# Prepare features and target
X = df.drop('resale_price', axis=1)
y = df['resale_price']

In [7]:
#Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Train Linear Regression Model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression()

In [9]:
# Train Decision Tree Model
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [10]:
# Evaluate both models
def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    print(f"\n{model_name} Performance:")
    print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")
    print(f"RMSE: {mean_squared_error(y_test, y_pred, squared=False):.2f}")
    print(f"R²: {r2_score(y_test, y_pred):.4f}")

evaluate_model(lr_model, X_test, y_test, "Linear Regression")
evaluate_model(dt_model, X_test, y_test, "Decision Tree")


Linear Regression Performance:
MAE: 48502.04
RMSE: 64025.74
R²: 0.8616

Decision Tree Performance:
MAE: 21654.77
RMSE: 33244.22
R²: 0.9627


In [11]:
# Function to predict future price
def predict_future_price(year, month, lease_commence_year, floor_area, storey_range, town, flat_type, flat_model):
    # Calculate lease remaining
    lease_remaining = 99 - (year - lease_commence_year)

    data = {
        'year': [year],
        'month': [month],
        'lease_remaining': [lease_remaining],
        'floor_area_sqm': [floor_area]
    }
    
    # Create DataFrame
    input_df = pd.DataFrame(data)
    for col in X_train.columns:
        if col not in input_df.columns:
            input_df[col] = 0
            
    # Set the specific features to 1
    input_df[f'town_{town}'] = 1
    input_df[f'flat_type_{flat_type}'] = 1
    input_df[f'flat_model_{flat_model}'] = 1
    input_df[f'storey_range_{storey_range}'] = 1
    
    # Ensure columns are in the same order
    input_df = input_df[X_train.columns]
    
    # Scale the numerical features
    input_df[['floor_area_sqm', 'lease_remaining']] = scaler.transform(input_df[['floor_area_sqm', 'lease_remaining']])
    
    lr_pred = lr_model.predict(input_df)[0]
    dt_pred = dt_model.predict(input_df)[0]
    
    return lr_pred, dt_pred

In [27]:
future_price_lr, future_price_dt = predict_future_price(
    year=2050,
    month=9,
    lease_commence_year=2028,
    floor_area=93,
    storey_range='16 TO 18',
    town='QUEENSTOWN',
    flat_type='4 ROOM',
    flat_model='Model A'
)

print(f"\nPredicted prices for 2034:")
print(f"Linear Regression: SGD {future_price_lr:,.2f}")
print(f"Decision Tree: SGD {future_price_dt:,.2f}")
print(f"Average Prediction: SGD {(future_price_lr + future_price_dt)/2:,.2f}")


Predicted prices for 2034:
Linear Regression: SGD 1,125,722.00
Decision Tree: SGD 770,000.00
Average Prediction: SGD 947,861.00
